[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jamescalam/applied-ml-minicourse/blob/main/code/13-hf-spaces-secrets.ipynb)

# 13: Hugging Face Spaces Secrets

As our app files will be publically accessible we need a way to hide our API keys and auth tokens. For the Pinecone API key and Hugging Face auth token we can do this easily by adding them to the app secrets:

TK screenshot

Then we access these values using the `os` module like so:

```python
PINECONE_KEY = os.environ['PINECONE_KEY']
HF_AUTH = os.environ['HF_AUTH']
```

Storing our *Cloud Storage* credentials is more difficult. For this we must encrypt the `cloud-storage.json` credentials file and store it in an accessible location. If encrypted correctly the information inside the encrypted file can only be understood when given a specific *decryption key* generated during the encryption process.

To encrypt the file (and generate the encryption key) we first load `cloud-storage.json`:

In [ ]:
import json

with open('cloud-storage.json', 'r', encoding='utf-8') as fp:
    creds = json.load(fp)

Convert to string format with `json.dumps`:

In [ ]:
keys_str = json.dumps(api, indent=4)

Now we initialize an encryption key using the `cryptography` module:

In [ ]:
from cryptography.fernet import Fernet

key = Fernet.generate_key()

Use the key to initialize encryption/decryption object:

In [ ]:
fernet = Fernet(key)

And encrypt the credentials string:

In [ ]:
encrypted = fernet.encrypt(keys_str.encode())

Now we save the encrypted credentials to file and add to the Hugging Face space.

In [ ]:
with open('cloud-storage.encrypted', 'wb') as fp:
    fp.write(encrypted)

To decrypt this file we *need* the same encryption key we have stored in the `key` variable. We copy it across to our Hugging Face spaces secrets.

TK screenshot of adding to app secrets

Back in our app code we can decrypt the encrypted credentials file and store them in a private location to be read by the Cloud Storage authenticator.

In [ ]:
with open('cloud-storage.encrypted', 'rb') as fp:
    encrypted = fp.read()
    creds = json.loads(fernet.decrypt(encrypted).decode())

Now `creds` contains our exact `cloud-storage.json` credentials, decrypted from the `cloud-storage.encrypted` file.

With all of these steps complete, we are ready to deploy our app to Hugging Face Spaces.